# Metabolic modelling of a threonine biosynthetsis pathway in E. coli: Optimisation of the threonine production
*Note: The following project is based on the Chapter 9 ("Metabolsm") of the textbook "Fundamentals of Systems Biology" by Markus W. Covert*
<br>
<br>

In [1]:
# Numerical & data modules
import numpy as np
import pandas as pd

# Optimisation module
import cvxpy as cvx

import os
import re

# Import Matplotlib
import matplotlib.pyplot as plt

# Import function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline


### L-threonine biosynthesis pathway analysis
We will try to analyze the threonine biosynthesis pathway in E. coli K12 strain. First, we will use a simpler way and we go to [Ecocyc](https://ecocyc.org) webpage, where we can simply (after registration) type "threonine" into the search field and then we can select "L-threonine biosynthesis". We will get the overview of the pathway in the form of schematics like this:
![Scheme of L-threonine biosynthesis pathway](L_threonine_biosynthesis.png)

From the above scheme, we can try to derive the reaction rate equations describing the dynamics of the two reactions governing the conversion of L-homoserine to L-threonine. For now, we ignore the exchange fluxes. Then the rate equations are as follows:

\begin{align}
\frac{d}{dt}[L-homoserine] &= - v_{thrB} \\
\frac{d}{dt}[ATP] &= - v_{thrB} \\
\frac{d}{dt}[ADP] &= v_{thrB} \\
\frac{d}{dt}[H] &= v_{thrB} \\
\frac{d}{dt}[O-phospho-L-homoserine] &= v_{thrB} - v_{thrC} \\
\frac{d}{dt}[H_{2}O] &= - v_{thrC} \\
\frac{d}{dt}[phosphate] &=  v_{thrC} \\
\frac{d}{dt}[L-threonine] &=  v_{thrC}
\end{align}

where $v_{thrB}$ and $v_{thrC}$ are the reaction rates of the reactions. If we denote the vector of all species' concentration as $[M]$ and the vector of the reaction rates as $v$, the reaction rate equations can be compactly written as

\begin{align}
\frac{d}{dt}[M] = Sv
\end{align}

where $S$ is a stoichiometric matrix. In our case, the reaction can be written as
\begin{align}
    \frac{d}{dt}\begin{bmatrix}
        [L-homoserine] \\
        [ATP] \\
        [ADP] \\
        [H] \\
        [O-phospho-L-homoserine] \\
        [H_{2}O] \\
        [phosphate] \\
        [L-threonine]
\end{bmatrix} &=
    \begin{bmatrix}
        -1 & 0 \\
        -1 & 0 \\
         1 & 0 \\
         1 & 0 \\
         1 & -1 \\
         0 & -1 \\
         0 &  1 \\
         0 &  1
    \end{bmatrix}
    \begin{bmatrix}
        v_{thrB} \\
        v_{thrC}
    \end{bmatrix}
\end{align}

### L-threonine production: exchange fluxes
We will now try to set up a linear optimization problem that will allow us to *determine the amount of L-threonine that can be produced from the input feed of 10 molecules/h for L-homoserine.* We will assume that all the reactants are available from the environment, except for O-phospho-L-homoserine.

In our approach, we will assume the steady state of the reaction, as we are considering the longer timescales (hours) than those compared to the reactions dynamics (seconds). To exploit the **Flux balance analysis** of the steady state reactions, we will include the **exchange fluxes** to allow us to work with the steady state description of the system:

\begin{align}
\frac{d}{dt}[L-homoserine] &= - v_{thrB} + b_{hms} = 0 \\
\frac{d}{dt}[ATP] &= - v_{thrB} + b_{ATP} \\
\frac{d}{dt}[ADP] &= v_{thrB} + b_{ADP} \\
\frac{d}{dt}[H] &= v_{thrB} + b_{H^{+}} \\
\frac{d}{dt}[O-phospho-L-homoserine] &= v_{thrB} - v_{thrC} + b_{phs} \\
\frac{d}{dt}[H_{2}O] &= - v_{thrC} + b_{H_{2}O} \\
\frac{d}{dt}[phosphate] &=  v_{thrC} + b_{phos} \\
\frac{d}{dt}[L-threonine] &=  v_{thrC} + b_{thr}
\end{align}

Therefore, the original system can be transformed into steady state form with the amended stoichiometric matrix $S_{b}$ and concatenated rate vector $v_{b}$ that contains not only the reaction rates $v_{thrB}$ and $v_{thrC}$, but also the exchange fluxes of all the reactants and products in the system:

\begin{align}
    S_{b}v_{b} &=
    \begin{bmatrix}
        -1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        -1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
         1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
         1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
         1 & -1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
         0 & -1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
         0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
         0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
    \end{bmatrix}
    \begin{bmatrix}
        v_{thrB} \\
        v_{thrC} \\
        b_{hms} \\
        b_{ATP} \\
        b_{ADP} \\
        b_{H^{+}} \\
        b_{phs} \\
        b_{H_{2}O} \\
        b_{phos} \\
        b_{thr}
    \end{bmatrix}
    = 0
\end{align}

where $b_{i}$ is the exchange flux of the i-th species.

### Setting up optimization problem for L-threonine production
The optimization problem can be set up as 

\begin{align}
\text{max} \ (-b_{thr}) \\
\text{subject to} \\
\ S_{b}v_{b} &= 0 \\
0 \leq v_{thrC} &\leq +\infty \\
0 \leq b_{hms} &\leq 10 \\
0 \leq b_{phs} &\leq 0
\end{align}

We try to *maximize* the negative of the exchange flux of L-threonine, as we are interested in the maximization of the flux *leaving* the system.

Looking at the problem further, the equality constraint describes the steady state rate equations, the first inequality constraint bounds the rate of production $v_{thrC}$ to positive values and hence implies irreversibility of the reactions, the second inequality constraint describes the admissible range of the flux of L-homoserine, and the last inequality constraint (which could also be written as equality constraint) arises from the fact that if initially, no O-phospho-L-homoserine is present in the system and hence under the steady-state assumption, all of this species produced must be also used up in the reactions. The units of all species will be molecules/h. From a mathematical standpoint, we have a standard linear optimization problem, which is also convex. So we will reach out to Stanford's Professor Stephen Boyd for help and we will use the marvelous `cvxpy` toolbox to solve this problem.

In [61]:
# Solves a bounded linear optimization problem

# Define the stoichiometric matrix
S_b = np.array(
      [[-1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [-1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [ 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [ 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [ 1, -1, 0, 0, 0, 0, 1, 0, 0,0],
       [ 0, -1, 0, 0, 0, 0, 0, 1, 0,0],
       [ 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [ 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]])

# Define array of zeros
zero_vals = np.zeros((8,1))

# Construct the problem

# Define the lower bounds
lower_bounds = np.ones((10,1))*(-(float('inf')))
lower_bounds[1] = 0 # v_thrC
lower_bounds[2] = 0 # b_hms
lower_bounds[6] = 0 # b_phs

# Define the upper bounds
upper_bounds = np.ones((10,1))*((float('inf')))
upper_bounds[2] = 10 # b_hms
upper_bounds[6] = 0 # b_phs


# Define the exchange fluxes vector
v_b = cvx.Variable((10,1))

# Define the exchange flux of L-threonine
b_thr = v_b[9]

# Define the objective function
objective = cvx.Minimize(b_thr)

# Define the constraints
constraints = [S_b*v_b == zero_vals,
               lower_bounds <= v_b,
               v_b <= upper_bounds]

# Problem definition
prob = cvx.Problem(objective, constraints)

print("Maximum L-threonine flux:", prob.solve())
print()
print("Rates: ")
print(v_b.value[:2])
print()
print("Exchange fluxes:")
print(v_b.value[2:])



Maximum L-threonine flux: -10.00257961780823

Rates: 
[[10.00201329]
 [10.00257962]]

Exchange fluxes:
[[ 1.00020122e+01]
 [ 1.00020133e+01]
 [-1.00020133e+01]
 [-1.00020133e+01]
 [ 5.65201780e-04]
 [ 1.00025796e+01]
 [-1.00025796e+01]
 [-1.00025796e+01]]


From the above results, we can see that the maximum L-threonine flux is 10 molecules/h. The flux of O-phospho-L-homoserine is almost zero (up to the convex optimization algorithm error).

# Flux Balance Analysis of the Production of Artemisinin
*(Problem from Covert, M. W. Fundamentals of Systems Biology, 2014*)

As a more intricate problem, we will analyze the simplified version of the antimalarial drug artemisinin, which was engineered by Jay Keasling at the UC Berkeley (Ro et al, 2006). Ro et al. engineered a pathway that produced intermediates, which could be converted to artemisinic acid, a precursor of artemisinin, through introduction of an amorphadiene synthase and a novel cytochrome (a protein that contains heme as a cofactor) P450 (CYP71). A simplified version of the target pathway is in the figure below (adopted from *Fundamentals of Systems Biology*):